In [24]:
import pandas as pd
from sqlalchemy import create_engine
from config import key
from sqlalchemy.ext.declarative import declarative_base

In [25]:
Base = declarative_base()

### Read in Constructor Results CSV

In [5]:
csv_file = "/Users/kscomputer/Desktop/Formula_ETL/Formula1ETLProject/Resources/constructorResults.csv"
constructorResults_df = pd.read_csv(csv_file)
constructorResults_df.head()

,constructorResultsId,raceId,constructorId,points,status
0,1,18,1,14.0,NaN
1,2,18,2,8.0,NaN
2,3,18,3,9.0,NaN
3,4,18,4,5.0,NaN
4,5,18,5,2.0,NaN


##### see if any values in status and drop if nothing

In [6]:
constructorResults_df['status'].value_counts()

D    17
Name: status, dtype: int64

### Read in Constructors.csv

In [7]:
csv_file2 = "/Users/kscomputer/Desktop/Formula_ETL/Formula1ETLProject/Resources/constructors.csv"
constructor_df = pd.read_csv(csv_file2)
constructor_df.head()

,constructorId,constructorRef,name,nationality,url,Unnamed: 5
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren,NaN
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber,NaN
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...,NaN
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_F1,NaN
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso,NaN


##### see if any values in unnamed: 5 and drop if nothing

In [8]:
constructor_df['Unnamed: 5'].unique()

array([nan])

##### drop unnamed: 5 because only nan values

In [9]:
constructor_df=constructor_df[['constructorId','constructorRef','name','nationality','url']].copy()

In [10]:
constructor_df.head()

,constructorId,constructorRef,name,nationality,url
0,1,mclaren,McLaren,British,http://en.wikipedia.org/wiki/McLaren
1,2,bmw_sauber,BMW Sauber,German,http://en.wikipedia.org/wiki/BMW_Sauber
2,3,williams,Williams,British,http://en.wikipedia.org/wiki/Williams_Grand_Pr...
3,4,renault,Renault,French,http://en.wikipedia.org/wiki/Renault_F1
4,5,toro_rosso,Toro Rosso,Italian,http://en.wikipedia.org/wiki/Scuderia_Toro_Rosso


### Read in Constructor Standings.csv

In [11]:
csv_file3 = "/Users/kscomputer/Desktop/Formula_ETL/Formula1ETLProject/Resources/constructorStandings.csv"
constructorstanding_df = pd.read_csv(csv_file3)
constructorstanding_df.head()

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins,Unnamed: 7
0,1,18,1,14.0,1,1,1,NaN
1,2,18,2,8.0,3,3,0,NaN
2,3,18,3,9.0,2,2,0,NaN
3,4,18,4,5.0,4,4,0,NaN
4,5,18,5,2.0,5,5,0,NaN


##### see if any values in unnamed: 7 and drop if nothing

In [12]:
constructorstanding_df['Unnamed: 7'].unique()

array([nan])

##### drop unnamed: 7 because only nan values

In [13]:
constructorstanding_df=constructorstanding_df[['constructorStandingsId','raceId','constructorId','points','position','positionText','wins']].copy()

In [14]:
constructorstanding_df.head()

,constructorStandingsId,raceId,constructorId,points,position,positionText,wins
0,1,18,1,14.0,1,1,1
1,2,18,2,8.0,3,3,0
2,3,18,3,9.0,2,2,0
3,4,18,4,5.0,4,4,0
4,5,18,5,2.0,5,5,0


### Read in races.csv

In [15]:
csv_file4 = "/Users/kscomputer/Desktop/Formula_ETL/Formula1ETLProject/Resources/races.csv"
races_df = pd.read_csv(csv_file4)
races_df.head()

,raceId,year,round,circuitId,name,date,time,url
0,1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...
1,2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...
2,3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...
3,4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...
4,5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...


#### connect to local databases

In [21]:
#this works on steven machine
engine = create_engine(f'sqlite:///Formula1')
conn = engine.connect()

In [27]:
Base.metadata.create_all(engine)

In [28]:
from sqlalchemy.orm import Session
session = Session(bind=engine)

In [29]:
session.add(constructorResults_df)
session.add(constructor_df)
session.add(constructorstanding_df)
session.add(races_df)
session.commit()

UnmappedInstanceError: Class 'pandas.core.frame.DataFrame' is not mapped

In [19]:
#works on raj machine
#rds_connection_string = f"postrges:password@localhost:5432/Formula1"
#engine = create_engine(f'postgresql://{rds_connection_string}')

#### check for tables

In [26]:
engine.table_names()

[]

In [ ]:
constructorResults_df.to_sql(name='ConstructorResults', con=engine, if_exists='append', index=False)

In [ ]:
constructor_df.to_sql(name='Constructor', con=engine, if_exists='append', index=False)

In [ ]:
constructorstanding_df.to_sql(name='ConstructorStandings', con=engine, if_exists='append', index=False)

In [ ]:
races_df.to_sql(name='Race', con=engine, if_exists='append', index=False)